<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/master/2%20Advanced/100MolecularOrbital.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google Colaboratory上で分子軌道計算を試します。

[Labo-code](https://labo-code.com/python/quantum-chemical-calculation/moleculaorbital/)の記事を参考にしました。感謝!


# 1. 初期化
## 1.1 必要なパッケージのインストール


In [ ]:
!pip install pyscf
!pip install numpy
!pip install geometric
!pip install py3Dmol

# 2. 構造最適化

## 2.1 初期データを作る

Labo-codeさんの記事をそのまま使っても構いませんが、ここではベンゼンではなく水分子を作ってみます。

水分子の正確な原子位置はわからない(むしろ計算した結果として得たい)ので、ベンゼンの炭素3つの座標を酸素と水素に使います。



In [ ]:
from pyscf import gto
# GTO: Gaussian-type Orbital


# 水分子の定義 (座標の単位はÅ、すごくてきとう。)
mol = gto.Mole()
mol.atom = """
O 0.0 0.0 0.0
H 1.0 0.0 0.0
H 0.0 1.0 0.0
"""
mol.basis='6-31G(d)'
mol.build()

In [ ]:
# とりあえず、そのまま表示してみる
import py3Dmol
from IPython.display import display, HTML


def show_mol(mol, viewer):
    # 分子のサイズを取得
    coords = mol.atom_coords()
    x_min, y_min, z_min = coords.min(axis=0) - 2.0
    x_max, y_max, z_max = coords.max(axis=0) + 2.0


    # 分子構造の追加
    xyz = mol.tostring(format="xyz")
    viewer.addModel(xyz, "xyz")
    viewer.setStyle({"stick": {}, "sphere": {"scale": 0.3}})


# xyz = mol_to_xyz(mol)
# view = py3Dmol.view(width=800, height=600)
# view = view_xyz(view, xyz)
# display(HTML(view._make_html()))

# 3D表示の準備
viewer = py3Dmol.view(width=800, height=600)
show_mol(mol, viewer)
viewer.zoomTo()
viewer.show()


## 2.2 構造最適化

与えられた原子核の初期配置に対し、電子を配置してエネルギーができるだけ低くなるように電子と原子核の配置を最適にします。これを構造最適化と呼びます。


In [ ]:
from pyscf.geomopt import geometric_solver
from pyscf import dft, scf

# # DFT計算のセットアップ
# mf = dft.RKS(mol)

# SCF計算のセットアップ
mf = scf.RHF(mol)

# 構造最適化
mol_eq = geometric_solver.optimize(mf, maxsteps=100)


In [ ]:
view = py3Dmol.view(width=800, height=600)
show_mol(mol_eq, view)
view.zoomTo()
view.show()


## 2.3 電子配置

電子を、エネルギーの低い軌道から順に入れていきます。酸素と水素で全部で10個の電子があるので、5つの軌道が埋まります。

エネルギーの単位をhartreeからeVに換算して表示します。

In [ ]:
# 軌道情報の(再)計算
mf.kernel()

# エネルギーをeVに変換
hartree_to_ev = 27.2114

mf.mo_occ, mf.mo_energy * hartree_to_ev


電子が入っている、もっともエネルギーの高い(最も外側にある)軌道のことをHOMO (Highest Occupied Molecular Orbital; 最高被占分子軌道、その一つ上の軌道のことをLUMO (Lowest Unoccupied Molecular Orbital; 最低空分子軌道)と呼びます。一番外側の軌道は、分子の形や大きさや反応性を決定づけるので、化学ではこれらの軌道に特に注目します。

水の場合、HOMOは5番目、LUMOは6番目の軌道です。

Pythonでは、数字を0から数えはじめるので、これらはそれぞれ4番目、5番目に相当します。

In [ ]:
import numpy as np

# HOMOとLUMOのインデックス
lumo_index = np.argwhere(mf.mo_occ == 0)[0][0]
homo_index = lumo_index - 1


## 2.4. 軌道の可視化

途中で、軌道の形をcube形式のファイルに書きだし、それを読みこんでいます。

In [ ]:
from pyscf import tools
import py3Dmol
from IPython.display import display, HTML
import ipywidgets as widgets

# 指定した軌道のCUBEファイルを生成する関数
def generate_cube_file(orbital_index, filename):
    tools.cubegen.orbital(mol, filename, mf.mo_coeff[:, orbital_index])

# 可視化する軌道のリストを作成
orbitals = {
    "LUMO+2": lumo_index + 2,
    "LUMO+1": lumo_index + 1,
    "LUMO": lumo_index,
    "HOMO": homo_index,
    "HOMO-1": homo_index - 1,
    "HOMO-2": homo_index - 2,
    "HOMO-3": homo_index - 3,
    "HOMO-4": homo_index - 4
}

# エネルギーをeVに変換
hartree_to_ev = 27.2114
orbital_energies = {name: mf.mo_energy[idx] * hartree_to_ev for name, idx in orbitals.items()}

# 可視化関数
def show_orbital(orbitals, orbital_name, view):
    orbital_index = orbitals[orbital_name]
    cube_filename = f'{orbital_name}.cube'

    generate_cube_file(orbital_index, cube_filename)

    with open(cube_filename) as f:
        cube = f.read()
    view.addVolumetricData(cube, 'cube', {'isoval': 0.02, 'color': 'red', 'opacity': 0.75})
    view.addVolumetricData(cube, 'cube', {'isoval': -0.02, 'color': 'blue', 'opacity': 0.75})

    # エネルギーのラベルを追加
    orbital_energy = orbital_energies[orbital_name]
    view.addLabel(f'{orbital_name}: {orbital_energy:.6f} eV', {'fontSize': 12, 'fontColor': 'black', 'backgroundColor': 'white', 'backgroundOpacity': 0.8, 'showBackground': True, 'position': {'x': 0, 'y': 0, 'z': 0}})


Google Colabの簡易フォーム機能を使い、メニューで軌道を選べるようにしました。

In [ ]:
# @title 軌道の可視化
orbital_name = "LUMO" # @param ["LUMO+2", "LUMO+1", "LUMO", "HOMO", "HOMO-1", "HOMO-2", "HOMO-3", "HOMO-4"]
# @

view = py3Dmol.view(width=800, height=600)
show_mol(mol_eq, view)
show_orbital(orbitals, orbital_name, view)
view.zoomTo()
view.show()


# 3. 相互作用の計算

水とCO2の相互作用エネルギーを計算してみます。



## 3.1 CO2を作る


In [ ]:
from pyscf import gto, scf
from pyscf.geomopt import geometric_solver

# --- CO2分子の構造最適化 ---
co2 = gto.Mole()
co2.atom = [
    ['C', (0.0, 0.0, 0.0)],
    ['O', (1.2, 0.0, 0.0)],
    ['O', (-1.2, 0.0, 0.0)],
]
co2.basis = '6-31g*' # 基底関数系
co2.build()

mf_co2 = scf.RHF(co2).run()
# pybernyを使って構造最適化を実行
co2_opt = geometric_solver.optimize(mf_co2)

co2_opt.atom_coords()

## 3.2 ２分子複合体を作る

2分子が近くにある配置`combined`を作ります。

In [ ]:
# 最適化されたCO2の座標と原子名を取得
atom_names_co2 = [atom[0] for atom in co2_opt._atom]
atom_coords_co2 = co2_opt.atom_coords()

# 水も同様
atom_names_h2o = [atom[0] for atom in mol_eq._atom]
atom_coords_h2o = mol_eq.atom_coords()

# 水をx方向に5 bohrずらす。
atom_coords_co2[:, 0] += 8

atom_names_h2o, atom_coords_h2o

In [ ]:
# 複合系の原子リストと座標を作成
# H2OとCO2の原子情報を結合
combined_atom = []
for i in range(3):
    combined_atom.append([atom_names_h2o[i], *atom_coords_h2o[i]])
for i in range(3):
    combined_atom.append([atom_names_co2[i], *atom_coords_co2[i]])

combined_atom

## 3.3 分子配置を見る

In [ ]:
mol_complex = gto.M(atom=combined_atom, basis='6-31g*', unit='bohr')

view = py3Dmol.view(width=800, height=600)
show_mol(mol_complex, view)
view.zoomTo()
view.show()


## 3.4 複合体のエネルギー計算

ここでは、原子が動いてほしくないので、構造最適化は行わず、エネルギー計算だけを行います。

In [ ]:
# 複合系のエネルギー計算
mf_complex = scf.RHF(mol_complex).run()
e_complex = mf_complex.e_tot # 複合系の全エネルギー
e_complex # in hartree

## 3.5 会合によるエネルギー収支

In [ ]:
e_h2o = mf.e_tot
e_h2o # in hartree

In [ ]:
e_co2 = mf_co2.e_tot
e_co2 # in hartree

In [ ]:
(e_complex - (e_h2o+e_co2)) * 2625.5 # in kJ/mol

1 hartreeは2625 kJ/mol、かなり巨大な値です。電子状態計算でこんな大きなエネルギーがでてくるのは、原子核とすべての電子の間のクーロンエネルギーを計算するせいです。これに対し、相互作用を計算する場合には、分子が近付いたことによるエネルギー利得(差分)しか扱わないので、ずいぶん桁が小さくなります。したがって、電子状態計算をかなり精密に行わないと、相互作用の値は信用できなくなってしまいます。

